In [286]:
import numpy as np
import pandas as pd
from datetime import datetime

In [287]:
# 读取数据源
def read2df(filename, sheet_name,header=1):
       return pd.read_excel(filename + '.xlsx',sheet_name=sheet_name,header=header)
Courses = read2df('1.3.1_高职课程设置表','1.3.1_高职课程设置表')
Teaching = read2df('2.1.1_高职授课表','2.1.1_高职授课表')
fullTimeTeacher = read2df('专兼职教师（状态数据）','专职')
partTimeTeacher = read2df('专兼职教师（状态数据）','兼职')

C:\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [288]:
# 筛选交通工程学院
## 定义一个筛选器：筛选交通工程学院
def queryJT(df,str):
       jt = '交通工程学院'
       return df[df[str]==jt]
Courses = queryJT(Courses,'开课单位名称*')
Teaching = queryJT(Teaching,'开课单位名称*')
fullTimeTeacher = queryJT(fullTimeTeacher,'院系')
partTimeTeacher = queryJT(partTimeTeacher,'部门')
Teach_Course = pd.merge(Teaching,Courses,how='left',on=['课程名称*'])

# Teach_Course：汇总教师授课表和课程设置表
      '授课教师工号*', '授课教师姓名*', '教师类别*', '专业名称*_x', '专业代码*_x', '专业层次*_x', '学制*_x',
       '培养方式*_x', '具体形式*_x', '生源类型*_x', '授课年级*', '课程名称*', '课程代码*_x', '授课学期*',
       '开课单位名称*_x', '授课任务*', '班级名称*', '教学工作量（学时）*', '线上教学（学时）*', '填报人_x',
       '专业名称*_y', '专业代码*_y', '专业层次*_y', '学制*_y', '培养方式*_y', '具体形式*_y',
       '生源类型*_y', '年级*', '课程代码*_y', '课程类型*', '课程属性*', '公共基础课*', '专业（技能）课*',
       '课程性质*', '开课学期*', '开课单位名称*_y', '是否校企合作开发课程*', '合作企业名称*', '精品在线开放课程*',
       '是否课证融通课程*', '是否网络教学课程*', '课程思政示范课*', '课程总学时（个）*', '其中，实践教学学时（个）*',
       '填报人_y'
       需要的信息：
       基础课：专业（技能）课*   专业基础课
       实践课：课程类型*           不为A类
       校外教师：

In [289]:
# 实践课加*
def add_xing(row):
       return row['课程名称*'] if row['课程类型*'] == 'A类' else '*' + row['课程名称*']
Teach_Course['课程名称*'] = Teach_Course.apply(add_xing,axis=1)

In [290]:
# 分类基础、实践
Teach_Course_base = Teach_Course[Teach_Course['专业（技能）课*']=='专业基础课']
Teach_Course_prac = Teach_Course[Teach_Course['课程类型*'] !='A类']

# Teach_Course_3:提取所需数据
授课教师姓名*、课程名称*（、加入）、专业名称*_x（、加入）、教学工作量（学时）*（求和）

In [291]:
# 授课教师姓名*、课程名称*（、加入）、专业名称*_x（、加入）、教学工作量（学时）*（求和）
def textjoin(x):
       return '、'.join(x)
def groupbyName(df):
       df =  df.groupby('授课教师姓名*').agg({
       '课程名称*':textjoin,
       '专业名称*_x':textjoin,
       '教学工作量（学时）*':'sum'})
       # 不丢弃索引的方式重设索引
       df.reset_index(drop=False,inplace=True)
       df.columns = ['姓名','讲授课程','所在专业','承担教学工作量']
       return df
Teach_Course_Main_gp = groupbyName(Teach_Course)
Teach_Course_prac_gp = groupbyName(Teach_Course_prac)
Teach_Course_base_gp = groupbyName(Teach_Course_base)

# mainTeach_all：教师信息总表
# 处理专兼职表

In [292]:
# 处理标题不一致问题
fullTimeTeacher.columns = ['序号', '部门', '姓名', '性别', '出生年月', '年龄', '政治面貌', '民族', '学历', '学位', '专业','是否有教师资格证', '职称', '备注', '入职时间']
# 类别 改成主讲教师
fullTimeTeacher.drop('序号',axis=1,inplace=True)
partTimeTeacher.columns = ['部门', '姓名', '性别', '出生年月', '学历', '学位', '工作单位', '职称', '主讲教师']
# 增加主讲教师
fullTimeTeacher['主讲教师'] ='校内专任'
partTimeTeacher['主讲教师'] ='校外兼课'
# 制作主讲总表:mainTeach_all
mainTeach_all = pd.concat([partTimeTeacher,fullTimeTeacher])
# 设置姓名为索引
mainTeach_all.set_index('姓名',inplace=True)
# 计算年龄 
mainTeach_all['出生年月'] = pd.to_datetime(mainTeach_all['出生年月'])
mainTeach_all['年龄'] = datetime.now().year - mainTeach_all['出生年月'].dt.year
# 空值填充：民族
mainTeach_all['民族'].fillna('汉族', inplace=True)



In [293]:
mainTeach_all.columns

Index(['部门', '性别', '出生年月', '学历', '学位', '工作单位', '职称', '主讲教师', '年龄', '政治面貌',
       '民族', '专业', '是否有教师资格证', '备注', '入职时间'],
      dtype='object')

# 合并mainTeach_all、Teach_Course

In [294]:
# 制作主讲教师表:mainTeach
def merge_mainTeach(df):
       df = pd.merge(mainTeach_all,df,how='left',on='姓名')
       df = df.dropna(subset=['讲授课程'])
       df['序号'] = df.index+1
       df.set_index('序号')
       return df
mainTeach = merge_mainTeach(Teach_Course_Main_gp)
pracTeach = merge_mainTeach(Teach_Course_prac_gp)
baseTeach = merge_mainTeach(Teach_Course_base_gp)

In [295]:

baseTeach.columns

Index(['姓名', '部门', '性别', '出生年月', '学历', '学位', '工作单位', '职称', '主讲教师', '年龄',
       '政治面貌', '民族', '专业', '是否有教师资格证', '备注', '入职时间', '讲授课程', '所在专业', '承担教学工作量',
       '序号'],
      dtype='object')

In [296]:
main_col =['序号','部门','姓名','性别','民族','年龄','学历', '学位','是否有教师资格证','专业','职称','讲授课程', '所在专业','主讲教师']
prac_col = ['序号','部门','姓名','性别','民族','年龄','学历', '学位','是否有教师资格证','专业','职称','讲授课程','工作单位','主讲教师']
base_col = ['序号','部门','姓名','性别','民族','年龄','学历', '学位','是否有教师资格证','专业','职称','讲授课程', '所在专业','主讲教师']
mainTeach = mainTeach[main_col]
pracTeach = pracTeach[prac_col]
baseTeach = baseTeach[base_col]

In [297]:
# todo 如何写入excel中
from openpyxl import load_workbook
path = '交通-2022-2023主讲教师.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
def write2sheet(sheet_name, df):
       df.to_excel(writer, sheet_name=sheet_name, startrow=2, index=False, header=False)
       writer.save()
       writer.close()

write2sheet( '专业课主讲教师一览表', mainTeach)
write2sheet('实践性课程教师一览表', pracTeach)
write2sheet('基础性课程主讲教师一览表', baseTeach)

AttributeError: property 'book' of 'OpenpyxlWriter' object has no setter